In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime as dt
import boto3
from io import BytesIO
import os
from dotenv import load_dotenv

plt.style.use('ggplot')

In [ ]:
# Загрузка переменных окружения
load_dotenv()

S3_BUCKET_NAME=os.getenv("S3_BUCKET_NAME")
AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")


In [ ]:
# определение функций для работы с s3
def upload_to_s3(df, file_name):
    s3_client = boto3.client(
        "s3",
        endpoint_url='https://storage.yandexcloud.net',
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    )
    
    with BytesIO() as buffer:
        df.to_parquet(buffer)
        buffer.seek(0)
        s3_client.upload_fileobj(buffer, S3_BUCKET_NAME, file_name)
    
    
def download_from_s3(file_name) -> pd.DataFrame:
    s3_client = boto3.client(
        "s3",
        endpoint_url='https://storage.yandexcloud.net',
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    )
    with BytesIO() as buffer:
        s3_client.download_fileobj(S3_BUCKET_NAME, file_name, buffer)
        buffer.seek(0)
        return pd.read_parquet(buffer)

## Разделение на треин и тест  
Можно пропустить, если уже делали сохранение результатов

In [2]:
data = pd.read_parquet("../research/bank_products_processed.parquet")

data.head()

,age,work_expirience,income,days_from_first_contract,days_from_premium,is_last_6m_contract,is_new_client,is_resident,has_bank_spouse,is_acc_actual,...,vip_status,potential_client,is_KHE,is_KAT,is_KFC,is_capital,is_barcelona,is_south,client_id,div_data
0,35,6,87218.10,3872,0.0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,1375586,2015-01-28
1,23,35,35548.74,4757,0.0,0,1,1,0,0,...,0,0,1,0,0,0,0,1,1050611,2015-01-28
2,23,35,122179.11,4757,0.0,0,1,1,0,0,...,0,0,1,0,0,0,0,1,1050612,2015-01-28
3,22,35,119775.54,4757,0.0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1050613,2015-01-28
4,23,35,101850.00,4757,0.0,0,1,1,0,0,...,0,0,1,0,0,0,0,0,1050614,2015-01-28


In [3]:
print(data['div_data'].min())
print(data['div_data'].max())

2015-01-28 00:00:00
2016-05-28 00:00:00


In [11]:
split_date = '2016-01-28'

train_data = data[data['div_data'] <= split_date]
test_data = data[data['div_data'] > split_date]



In [19]:
upload_to_s3(train_data, "train_data.parquet")
upload_to_s3(test_data, "test_data.parquet")

## Отбор признаков

In [20]:
train_data = download_from_s3("train_data.parquet")
test_data = download_from_s3("test_data.parquet")


In [ ]:
target_cols = [col for col in train_data.columns if col.startswith('acc_')]